In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='4'

In [4]:
pip install PyDrive

Note: you may need to restart the kernel to use updated packages.


In [5]:
# pip install torch torchvision torchaudio
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install torchvision --upgrade --index-url https://download.pytorch.org/whl/cu118
# !pip install --upgrade torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install --force-reinstall torch torchvision --index-url https://download.pytorch.org/whl/cu118

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.4.1%2Bcu118-cp310-cp310-linux_x86_64.whl (857.6 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.19.1%2Bcu118-cp310-cp310-linux_x86_64.whl (6.3 MB)
  Using cached https://download.pytorch.org/whl/filelock-3.13.1-py3-none-any.whl (11 kB)
  Using cached https://download.pytorch.org/whl/typing_extensions-4.9.0-py3-none-any.whl (32 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached https://download.pytorch.org/whl/networkx-3.2.1-py3-none-any.whl (1.6 MB)
  Using cached https://download.pytorch.org/whl/Jinja2-3.1.3-py3-none-any.whl (133 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2024.2.0-py3-none-any.whl (170 kB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_cuda_nvrtc_cu11-11.8.89-py3-none

In [11]:
!pip install opencv-python
!pip install opencv-python-headless

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 60.5 MB/s eta 0:00:00a 0:00:01
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 44.3 MB/s eta 0:00:0000:0100:01


In [12]:
!pip install torchvision

Defaulting to user installation because normal site-packages is not writeable


In [7]:
!pip install google-api-core==2.11.0 google-api-python-client==2.79.0 google-auth==2.16.1 google-auth-httplib2==0.1.0 google-auth-oauthlib==0.4.1 googleapis-common-protos==1.58.0

Defaulting to user installation because normal site-packages is not writeable
  Using cached google_auth_oauthlib-0.4.1-py2.py3-none-any.whl.metadata (2.4 kB)
Using cached google_auth_oauthlib-0.4.1-py2.py3-none-any.whl (18 kB)
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.2.1
    Uninstalling google-auth-oauthlib-1.2.1:
      Successfully uninstalled google-auth-oauthlib-1.2.1
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [8]:
!pip install --upgrade google-auth-oauthlib

Defaulting to user installation because normal site-packages is not writeable
  Using cached google_auth_oauthlib-1.2.1-py2.py3-none-any.whl.metadata (2.7 kB)
Using cached google_auth_oauthlib-1.2.1-py2.py3-none-any.whl (24 kB)
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.1
    Uninstalling google-auth-oauthlib-0.4.1:
      Successfully uninstalled google-auth-oauthlib-0.4.1
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [9]:
pip install requests google-auth-oauthlib google-auth-httplib2

Note: you may need to restart the kernel to use updated packages.


In [16]:
import requests
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os

# OAuth 2.0 클라이언트 ID를 사용하여 인증
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

# OAuth 2.0 인증 설정
def authenticate():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # flow = InstalledAppFlow.from_client_secrets_file(
            #     'client_secrets.json', SCOPES)
            # creds = flow.run_local_server(port=0)
            auth_url, _ = flow.authorization_url(prompt='consent')
            print(f'Please go to this URL: {auth_url}')

            # 사용자가 브라우저에서 인증 후, 제공된 코드를 콘솔에 입력하도록 설정
            code = input('Enter the authorization code: ')
            creds = flow.fetch_token(code=code)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

# Google Drive API 호출
def list_files(creds):
    url = 'https://www.googleapis.com/drive/v3/files'
    headers = {'Authorization': f'Bearer {creds.token}'}
    response = requests.get(url, headers=headers)
    return response.json()

creds = authenticate()
files = list_files(creds)
print(files)


{'nextPageToken': '~!!~AI9FV7RLC2PDURY3GIr9bX5yxf0KPER3ZcfXx5jIsRbkepGrXpxxtEF1SrP2zQaeiIM1B9Ajo0SP4JeIDB0Mrc3QAd1B-Z4fP7-KRq0tzVkDeQVXPZhmxEdcqSYkP_X-Y4sKNgNqIhKIAt76rfJyhdAmDik6joo2mSK4JjX-sLGgkPH49MzfznUdzRqa1KWV8NMHR5BITavPyxPzuv6kjfC8pWFV--wuYkgNVZ1Uvp7WMeeozRj29EBgBd4EKUSDSc4wr5WI4884eZkfnleCp0XeA6noJL-rtjlUMkGXLWp_NT13d0zO8yp1LEZQhAXh0tH7UknUyDjyitLHauXoEwT3IIlPHp-1PxYHPfRB3mLAX1H8rgJjO3Aqe6saXFQVx-6Z6emWhO_o6I4nDrWc08NszzmgLEFooHMR_EiB9nTRxPUbIS6JB8N78WaQ1cN2GkP_HwXm0OHVZINvLzqgcerChso0_K3zxhDobZCqXGGErzWsh2M18j8M7uZHVJ6D8--B-YsJmLBlNV7OOmWXGZBylxDD5JxG_4hrdR9qb6UZOUSxpAE9chI=', 'kind': 'drive#fileList', 'incompleteSearch': False, 'files': [{'kind': 'drive#file', 'mimeType': 'application/vnd.jgraph.mxfile', 'id': '19MhGtt2_sKhioPAx0tY6r1n3pS1alnYT', 'name': 'ssafy.drawio'}, {'kind': 'drive#file', 'mimeType': 'application/vnd.google-apps.presentation', 'id': '1pYgPLGPo1OOTeaswoVEKE4PPbNRtrwtxo0iCeObK_3U', 'name': '(특화)서울 3반'}, {'kind': 'drive#file', 'mimeType': 'image/jpeg', 'id

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os
# Google OAuth 인증 절차
gauth = GoogleAuth()
gauth.CommandLineAuth()  # 이 과정을 통해 브라우저가 열리고, Google 계정 인증이 이루어짐

# gauth.SaveCredentialsFile("credentials.json")
drive = GoogleDrive(gauth)


Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=20853731651-6grrfq6n7htqhferl1ah8imldetrgg53.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code



Enter verification code:  4/1AQlEd8yXnePXLqVPmZGQU6SWfpPZdHNMOJAtcRn8dbtkQOTL8ahbHjOQhpw


Authentication successful.


In [39]:
# import os
# folder_id = '1Nby_3Fg7NZ-CpLDjimVm9cC3n3imW4Sd'
# file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

# for file in file_list[:10]:
#     print(f"Downloading {file['title']} from {file['id']}")

In [ ]:
# 파일을 다운 받지않고 바로 메모리에서 처리하기!!!!!!!!! -> io.BytesIO 사용하기
# Google Drive에서 이미지와 라벨 파일을 다운로드하여 로컬 경로로 저장
def download_drive_folder(folder_id, local_path):
    file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()
    if not os.path.exists(local_path):
        os.makedirs(local_path)
    
    for file in file_list:
        if file['mimeType'] == 'application/vnd.google-apps.folder':  # 하위 폴더인 경우
            subfolder_local_path = os.path.join(local_path, file['title'])
            print(f"Entering folder: {file['title']}")
            download_drive_folder_recursive(file['id'], subfolder_local_path)  # 재귀적으로 하위 폴더 처리
        else:
            print(f"Downloading {file['title']} from Google Drive")
            file.GetContentFile(os.path.join(local_path, file['title']))

# Google Drive에서 폴더 ID를 이용해 데이터 다운로드
image_folder_id = '1Nby_3Fg7NZ-CpLDjimVm9cC3n3imW4Sd'  # Google Drive의 이미지 폴더 ID 입력
label_folder_id = '17KDAGcw4JLD3COQHQVYk68AyxUysTGPA'  # Google Drive의 라벨 폴더 ID 입력

image_local_path = './images'
label_local_path = './labels'

# 이미지 및 라벨 다운로드
download_drive_folder(image_folder_id, image_local_path)
download_drive_folder(label_folder_id, label_local_path)

# LaneDataset 클래스 정의
class LaneDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.image_folders = os.listdir(image_dir)
        self.label_folders = os.listdir(label_dir)

        # 각 폴더 안에 있는 이미지 파일들을 미리 저장
        self.image_files = []
        for folder in self.image_folders:
            folder_path = os.path.join(self.image_dir, folder)
            images = os.listdir(folder_path)  # 이미지 파일 리스트
            for img_file in images:
                self.image_files.append((folder, img_file)) # (폴더명, 이미지 파일명) 저장

    def __len__(self):
        return len(self.image_files)  # 모든 이미지 파일의 총 개수

    def __getitem__(self, idx):
        # 이미지 파일 정보
        folder, img_file = self.image_files[idx]
        img_path = os.path.join(self.image_dir, folder, img_file)

        # 해당 이미지 파일에 대응하는 라벨 폴더 및 라벨 파일
        label_folder_path = os.path.join(self.label_dir, folder)
        label_file = os.listdir(label_folder_path)[0]  # 라벨 파일 선택
        label_path = os.path.join(label_folder_path, label_file)

        # 이미지 로드
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 라벨 로드 (JSON 파일에서 차선 종류 및 위반 여부 정보 추출)
        with open(label_path, 'r') as f:
            label_data = json.load(f)

        # 위반 여부 및 차선 종류 추출
        violation, lane_type = self.extract_violation(label_path)

        # 필요한 경우 데이터 변환 적용
        if self.transform:
            image = self.transform(image)

        return image, violation, lane_type

    # JSON 파일에서 위반 여부와 차선 종류 추출 함수
    def extract_violation(self, json_file_path):
        with open(json_file_path, 'r') as f:
            data = json.load(f)

        # 위반 여부와 차선 종류 기본값 설정
        violation = False  # 기본적으로는 위반 아님
        lane_type = None   # 차선 종류는 None을 기본값으로 설정

        # "data_set_info"에서 "data" 항목에 있는 객체 정보 확인
        for obj in data['data_set_info']['data']:
            object_label = obj['value']['object_Label']

            # 위반 여부 판단
            if 'vehicle_attribute' in object_label and object_label['vehicle_attribute'] == 'violation':
                violation = True  # 위반 차량 있음
            # 차선 종류 판단
            if 'lane_type' in object_label:
                lane_type = object_label['lane_type']

        return violation, lane_type

# 데이터 변환 설정
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),  # ResNet 입력 크기와 맞추기
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 이미지와 라벨 데이터셋 생성
dataset = LaneDataset(image_dir=image_local_path, label_dir=label_local_path, transform=data_transform)
data_loader = DataLoader(dataset, batch_size=16, shuffle=True)

# DataLoader에서 데이터 순차적으로 불러오기
for images, (violations, lane_types) in data_loader:
    print(images.shape, violations, lane_types)

In [161]:
# !google auth oauthlib --version
!torch --version

/bin/bash: torch: command not found


In [75]:
!jupyter labextension install @jupyterlab/google-drive

(Deprecated) Installing extensions with the jupyter labextension install command is now deprecated and will be removed in a future major version of JupyterLab.

Users should manage prebuilt extensions with package managers like pip and conda, and extension authors are encouraged to distribute their extensions as prebuilt packages 
/opt/tljh/user/lib/python3.10/site-packages/jupyterlab/debuglog.py:54: UserWarning: An error occurred.
  warnings.warn("An error occurred.")
/opt/tljh/user/lib/python3.10/site-packages/jupyterlab/debuglog.py:55: UserWarning: ValueError: Please install Node.js and npm before continuing installation. You may be able to install Node.js from your package manager, from conda, or directly from the Node.js website (https://nodejs.org).
  warnings.warn(msg[-1].strip())
/opt/tljh/user/lib/python3.10/site-packages/jupyterlab/debuglog.py:56: UserWarning: See the log file for details: /tmp/jupyterlab-debug-5qghd27m.log
  warnings.warn(f"See the log file for details: {log

In [ ]:
# token 설정과 관련된 거 였는데, token 설정 잘 됨!
import requests
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os

# OAuth 2.0 클라이언트 ID를 사용하여 인증
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

# OAuth 2.0 인증 설정
def authenticate():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('client_secrets.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

# 최상위 폴더의 파일 및 폴더 목록을 가져오기 위한 함수
def list_root_files(creds, folder_id):
    url = f'https://www.googleapis.com/drive/v3/files?q=\'{folder_id}\' in parents'
    headers = {'Authorization': f'Bearer {creds.token}'}
    params = {
        'pageSize': 100,  # 가져올 최대 파일 수
        'fields': 'nextPageToken, files(id, name, mimeType)',  # 파일 이름, ID, MIME 타입
    }
    response = requests.get(url, headers=headers, params=params)
    return response.json()

# 폴더 ID 설정 (해당 폴더의 ID를 사용)
folder_id = '1nor9jn1J30mZN_168SsNj1ArO4duu1VT'  # Colab Notebooks/SpecializedProject/content/차로 위반 영상 데이터/sample_datas 
# 이 안에 라벨링데이터/TL/A/BLUE 안에 여러 폴더 + 원천데이터/TS1/A/BLUE 안에 여러 폴더

creds = authenticate()
files = list_root_files(creds, folder_id)
print('폴더 ID까지 설정 완료', files)
# 최상위 폴더에 있는 파일 및 폴더 출력
for file in files.get('files', []):
    print(f"Name: {file['name']}, ID: {file['id']}, MIME Type: {file['mimeType']}")


In [13]:
# 파일을 다운 받지않고 바로 메모리에서 처리하기
import io
import json
import requests
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
# import torch

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import cv2
import numpy as np

# OAuth 2.0 인증 설정 (기존 코드와 동일)
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

def authenticate():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('client_secrets.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

# Google Drive 파일을 메모리에서 읽는 함수
def download_drive_file(file_id, creds):
    url = f'https://www.googleapis.com/drive/v3/files/{file_id}?alt=media'
    headers = {'Authorization': f'Bearer {creds.token}'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return io.BytesIO(response.content)
    else:
        raise Exception(f"Error downloading file {file_id}, status code: {response.status_code}")

# 이미지와 라벨 데이터셋 정의 (메모리에서 처리)
class LaneDataset(Dataset):
    def __init__(self, image_file_ids, label_file_ids, creds, transform=None):
        self.image_file_ids = image_file_ids
        self.label_file_ids = label_file_ids
        self.transform = transform
        self.creds = creds

    def __len__(self):
        return len(self.image_file_ids)  # 모든 이미지 파일의 총 개수

    def __getitem__(self, idx):
        # Google Drive에서 이미지를 메모리로 다운로드
        img_file_id = self.image_file_ids[idx]
        img_data = download_drive_file(img_file_id, self.creds)
        image = np.frombuffer(img_data.read(), np.uint8)
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_GR2RGB)

        # Google Drive에서 라벨 데이터를 메모리로 다운로드
        label_file_id = self.label_file_ids[idx]
        label_data = download_drive_file(label_file_id, self.creds)
        label_json = json.load(label_data)

        # 위반 여부 및 차선 종류 추출
        violation, lane_type = self.extract_violation(label_json)

        # 필요한 경우 데이터 변환 적용
        if self.transform:
            image = self.transform(image)

        return image, violation, lane_type

    # JSON 파일에서 위반 여부와 차선 종류 추출 함수
    def extract_violation(self, label_data):
        violation = False  # 기본적으로는 위반 아님
        lane_type = None   # 차선 종류는 None을 기본값으로 설정

        for obj in label_data['data_set_info']['data']:
            object_label = obj['value']['object_Label']
            if 'vehicle_attribute' in object_label and object_label['vehicle_attribute'] == 'violation':
                violation = True
            if 'lane_type' in object_label:
                lane_type = object_label['lane_type']

        return violation, lane_type

# 데이터 변환 설정
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),  # ResNet 입력 크기와 맞추기
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Google Drive에서 가져올 이미지와 라벨 파일 ID 목록
image_file_ids = ['image_file_id_1', 'image_file_id_2']  # 실제 이미지 파일 ID 입력
label_file_ids = ['label_file_id_1', 'label_file_id_2']  # 실제 라벨 파일 ID 입력

creds = authenticate()

# 이미지와 라벨 데이터셋 생성
dataset = LaneDataset(image_file_ids=image_file_ids, label_file_ids=label_file_ids, creds=creds, transform=data_transform)
data_loader = DataLoader(dataset, batch_size=16, shuffle=True)

# DataLoader에서 데이터 순차적으로 불러오기
for images, (violations, lane_types) in data_loader:
    print(images.shape, violations, lane_types)


Exception: Error downloading file image_file_id_2, status code: 404

In [2]:
!pip show torch
!pip show torchvision

Name: torch
Version: 2.4.1+cu118
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /home/j-j11a303/.local/lib/python3.10/site-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu11, nvidia-cuda-cupti-cu11, nvidia-cuda-nvrtc-cu11, nvidia-cuda-runtime-cu11, nvidia-cudnn-cu11, nvidia-cufft-cu11, nvidia-curand-cu11, nvidia-cusolver-cu11, nvidia-cusparse-cu11, nvidia-nccl-cu11, nvidia-nvtx-cu11, sympy, triton, typing-extensions
Required-by: torchaudio, torchvision
Name: torchvision
Version: 0.19.1+cu118
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: /home/j-j11a303/.local/lib/python3.10/site-packages
Requires: numpy, pillow, torch
Required-by: 
